Use the precalculated output from resnet50 and train a Dense (FC) model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bcolz

from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import applications

In [3]:
def load_array(fname):
    return bcolz.open(fname)[:]

In [4]:
train_conv_layers = load_array('./data/train_resnet50_conv.arr')
valid_conv_layers = load_array('./data/valid_resnet50_conv.arr')
train_labels = load_array('./data/train_labels.arr')
valid_labels = load_array('./data/valid_labels.arr')

In [5]:
train_conv_layers.shape, train_labels.shape, valid_conv_layers.shape, valid_labels.shape

((3025, 2048), (3025, 8), (752, 2048), (752, 8))

In [6]:
inp = layers.Input(shape=(2048,), dtype='float32')

In [7]:
p = 0.2
x = layers.Dense(512, activation='relu')(inp)
x = layers.BatchNormalization()(x)
x = layers.Dropout(p)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(p / 2)(x)
x = layers.Dense(8, activation='softmax')(x)

In [8]:
model = models.Model(inp, x)

In [9]:
model.compile(optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_conv_layers, train_labels, validation_data=(valid_conv_layers, valid_labels), batch_size=64, epochs=10) 

After 10 epochs we got `loss: 0.0693 - acc: 0.9779 - val_loss: 0.3008 - val_acc: 0.9269`

## Submission

In [12]:
import submission

In [14]:
test_conv_layers = load_array('./data/test_resnet50_conv.arr')

In [17]:
test_filenames = load_array('./data/test_resnet50_filenames.arr')

In [19]:
preds = model.predict(test_conv_layers)

In [21]:
submission.gen_file(preds=preds, filenames=test_filenames)